## Ceneo Scraper

## Struktura pojedynczej opinii
|Składowa|Selektor|zmienna|
|--------|--------|-------|
|id opinii|[data-entry-id]| opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation|recommendation|
|gwiazdki|span.user-post__score-count|stars|
|treść|div.user-post__text|contents|
|lista zalet|div.review-feature__title--positives ~ div.review-feature_item|pros|
|lisita wad|div.review-feature__title--negatives ~ div.review-feature_item|cons| 
|dla ilu przydatna|button.vote-yes > span|helpful|
|dla ilu nieprzydatna| button.vote-no > span|unhelpful|
|data wystawienia|span.user-post_published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post_published > time:nth-child(2)["datetime"]|purchase_date|


## załadowanie bibliotek


In [50]:
import requests
from bs4 import BeautifulSoup
import json
import os

In [51]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                    return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return 

## Selektory składowych opinii

In [52]:
selectors= {
    'opinion_id' : (None, "data-entry-id"),
    'author' : ( "span.user-post__author-name",),
    'recommendation' : ("span.user-post__author-recomendation",),
    'stars' : ("span.user-post__score-count",),
    'contents' : ("div.user-post__text",),
    'pros' : ("div.review-feature__title-review-feature__title--positives ~ div.review-feature__item", None, True),
    'cons' : ("div.review-feature__title-review-feature__title--negatives ~ div.review-feature__item", None, True),
    'helpful' : ("button.vote-yes > span",),
    'unhelpful' : ("button.vote-no > span",),
    'publish_date' :("span.user-post__published > time:nth-child(1)", "datetime"),
    'purchase_date' :  ("span.user-post__published > time:nth-child(2)","datetime")
}

## Adres URL pierwszej strony z opiniami

In [53]:
product_id = "39562616"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

## Pobieranie wszystkich opinii o produkcie 

In [54]:
all_opinions = []
while(url):
    response = requests.get(url)
    response.status_code
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    opinion = opinions.pop(0)
    for opinion in opinions:
        single_opinion= {
            key: extract(opinion, *value)
            for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl/"+ page_dom.select_one("a.pagination__next")["href"].strip()
    except TypeError:
        url = None

## Zapis opinii do pliku JSON

In [55]:
page_dom = BeautifulSoup(response.text , 'html.parser')

In [56]:
if not os.path.exists("opinions"):
    os.makedirs("opinions")
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions,jf ,  indent=4, ensure_ascii=False)